# remove high frequency last words from Patent View nad CRSP company names after cleaning

input files: rawpv_clean_26Apr2020.csv, crsp_clean_26Apr20.csv
output files: pv_disc_26Apr20.csv, crsp_disc_26Apr20.csv

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import  re

import warnings
warnings.filterwarnings('ignore')


In [2]:
pv = pd.read_csv("rawpv_clean_26Apr2020.csv",encoding='latin-1')

crsp = pd.read_csv("crsp_clean_26Apr20.csv",encoding='latin-1')

In [3]:
pv['org_first'] = pv['orglow9'].apply(lambda x: x[0])

crsp['com_first'] = crsp['comlow4'].apply(lambda x: x[0])


In [4]:
pv['org_first'].unique()

array(['u', 'x', 'c', 'm', 'l', 'j', 'd', 'n', 's', 'h', 'k', 'q', 'r',
       'p', 'f', 'g', 't', 'i', 'v', 'a', 'w', 'e', 'b', 'o', 'y', '2',
       'z', '3', '8', '1', '7', '5', '9', '`', '4', '0', '6', '@', '?',
       '_', '#', '[', '!', '=', '(', '|', '{', ':', '*', '<', ';'],
      dtype=object)

In [5]:
lst = ['@', '?','`',
       '_', '#', '[', '!', '=', '(', '|', '{', ':', '*', '<', ';']

for i in lst:
    
    pv['orglow9'] = pv['orglow9'].str.replace(i,"")

In [6]:
pv.columns

Index(['patent_id', 'assignee_id', 'type', 'organization', 'sequence',
       'bractext', 'organization2', 'organization3', 'bractext1', 'orglow',
       'orglow2', 'orglow3', 'orglow4', 'orglow5', 'orglow6', 'last',
       'orglow7', 'orglow8', 'orglow9', 'org_first'],
      dtype='object')

In [7]:
crsp.columns

Index(['PERMNO', 'COMNAM', 'PERMCO', 'crsp_year', 'COMNAM2', 'comlow',
       'comlow2', 'last', 'len', 'comlow3', 'comlen', 'last2', 'comlow4',
       'com_first'],
      dtype='object')

In [8]:
pv2 = pv[['patent_id', 'assignee_id', 'organization', 'orglow9', 'org_first']]

In [9]:
crsp2 = crsp[['PERMNO', 'COMNAM', 'PERMCO', 'comlow4','com_first']]

In [10]:
crsp2['comlow_new'] = crsp2['comlow4']
pv2['orglow_new'] = pv2['orglow9']

In [44]:
c1 = crsp2.drop_duplicates('comlow4',keep = 'first')
p1 = pv2.drop_duplicates('orglow9',keep = 'first')

In [50]:
l1 =  list(p1['last'])

In [51]:
l2 = list(c1['last'])

In [52]:
lst = list(l1 + l2)

In [53]:
hfq = pd.DataFrame({'lst':lst})

In [56]:
z = hfq['lst'].value_counts()

In [80]:
z[(z>33)&(z<=35)].index.sort_values()

Index(['academy', 'aktiengellschaft', 'antriebstechnik', 'architects', 'b',
       'blue', 'bridge', 'building', 'commercial', 'composite', 'connections',
       'consolidated', 'copper', 'crafts', 'detection', 'dimensions',
       'fashions', 'forming', 'fusion', 'growers', 'heating', 'housewares',
       'invent', 'jidosha', 'kosan', 'montreal', 'mortgage', 'munchen',
       'nuclear', 'pen', 'play', 'quest', 'robot', 'saver', 'search', 'soft',
       'studies', 'track', 'umwelttechnik', 'valley', 'verfahrenstechnik',
       'virginia', 'western', 'wind'],
      dtype='object')

In [185]:
hifq =   ['america', 'communication', 'communications',
       'company', 'corp', 'corporation', 'corportion', 'de',
       'deutschland', 'development', 'electric', 'electronic',
       'electronics', 'enterprises', 'for', 'gesellschaft', 'global',
       'group', 'groups', 'holding', 'holdings','industrial',
       'industries', 'industry', 'international', 'japan', 'kabushiki',
       'la', 'lm', 'ltd', 'manufacturing', 'motor', 'national', 'network',
       'networks', 'new england', 'new holland', 'north american', 'of', 'american',
       'pharmaceuticals', 'pharmaceutical','technologies', 'product',
       'products', 'service', 'services', 'system', 'systems', 'tech',
       'technologies', 'technology', 'the', 'tokyo', 'und', 
       'united states', 'united','us', 'works', 'worldwide']

In [187]:
for i in hifq:
    j = " "+i+" "
    y = " "+i+"$"
    z = "^" + i +" "
    crsp2.loc[crsp2['comlow4'].str.contains(i),'comlow_new'] = crsp2[crsp2['comlow4'].str.contains(i)]['comlow_new'].str.replace(j," ")
    pv2.loc[pv2['orglow9'].str.contains(i),'orglow_new'] = pv2[pv2['orglow9'].str.contains(i)]['orglow_new'].str.replace(j," ")
    crsp2.loc[crsp2['comlow4'].str.endswith(i),'comlow_new'] = crsp2[crsp2['comlow4'].str.endswith(i)]['comlow_new'].str.replace(y,"")
    pv2.loc[pv2['orglow9'].str.endswith(i),'orglow_new'] = pv2[pv2['orglow9'].str.endswith(i)]['orglow_new'].str.replace(y,"")
    crsp2.loc[crsp2['comlow4'].str.startswith(i),'comlow_new'] = crsp2[crsp2['comlow4'].str.startswith(i)]['comlow_new'].str.replace(z,"")
    pv2.loc[pv2['orglow9'].str.startswith(i),'orglow_new'] = pv2[pv2['orglow9'].str.startswith(i)]['orglow_new'].str.replace(z,"")
    

In [188]:
lst = ['labs', 'lab','laboratories','laboratory','intellectual property',
      'engineering','energy', 'products','internationale','solutions','therapeutics'
       ]
for i in lst:
    y = " "+i+"$"
    crsp2.loc[crsp2['comlow_new'].str.endswith(i),'comlow_new'] = crsp2[crsp2['comlow_new'].str.endswith(i)]['comlow_new'].str.replace(y,"")
    pv2.loc[pv2['orglow_new'].str.endswith(i),'orglow_new'] = pv2[pv2['orglow_new'].str.endswith(i)]['orglow_new'].str.replace(y,"")

In [189]:
lst = ['corp', 'corporation','technology','technologies']

for i in lst:
    y = i+"$"
    crsp2.loc[crsp2['comlow_new'].str.endswith(i),'comlow_new'] = crsp2[crsp2['comlow_new'].str.endswith(i)]['comlow_new'].str.replace(y,"")
    pv2.loc[pv2['orglow_new'].str.endswith(i),'orglow_new'] = pv2[pv2['orglow_new'].str.endswith(i)]['orglow_new'].str.replace(y,"")

In [190]:
crsp2['comlow_new'] = crsp2['comlow_new'].apply(lambda x: x.strip())
pv2['orglow_new'] = pv2['orglow_new'].apply(lambda x: x.strip())

In [192]:
crsp2.loc[crsp2['comlow_new']=="","comlow_new"] = crsp2.loc[crsp2['comlow_new']=="","comlow4"]
pv2.loc[pv2['orglow_new']=="","orglow_new"] = pv2.loc[pv2['orglow_new']=="","orglow9"]

In [193]:
crsp2.to_csv("crsp_totest.csv", index = False)


In [195]:
crsp2['com_ns'] = crsp2['comlow4'].apply(lambda x: x.replace(" ",""))

pv2['org_ns'] = pv2['orglow9'].apply(lambda x: x.replace(" ",""))

In [196]:
cleanmerge = pd.merge(crsp2, pv2, left_on = ['com_ns'], right_on = ['org_ns'], how = "outer")

In [198]:
cleanmerge = cleanmerge[~(cleanmerge['com_ns'].isna())]
cleanmerge = cleanmerge[~(cleanmerge['org_ns'].isna())]

In [199]:
kpss = pd.read_csv("new_kpss_clean_11Apr2020.csv") #6453 are common")

In [200]:
len(set(cleanmerge['PERMNO'].unique()) &  set(kpss['permno'].unique())) #6453

6453

In [202]:
pno = list(set(kpss['permno'].unique()) - set(cleanmerge['PERMNO'].unique())) #1083

In [203]:
len(set(pno) & set(crsp2.PERMNO.unique() ))  ##636 more to match

636

In [204]:
pno2 = list(set(pno) & set(crsp2.PERMNO.unique() ))

In [205]:
ctm = crsp2[crsp2['PERMNO'].isin(pno2)]

In [207]:
ctm = ctm.drop_duplicates(keep = 'first')

In [209]:
ctm.to_csv("ctm_new.csv", index = False)

In [210]:
import time

In [213]:
pv2.org_first.unique()

array(['u', 'x', 'c', 'm', 'l', 'j', 'd', 'n', 's', 'h', 'k', 'q', 'r',
       'p', 'f', 'g', 't', 'i', 'v', 'a', 'w', 'e', 'b', 'o', 'y', '2',
       'z', '3', '8', '1', '7', '5', '9', '`', '4', '0', '6', '@', '?',
       '_', '#', '[', '!', '=', '(', '|', '{', ':', '*', '<', ';'],
      dtype=object)

In [27]:
pv3.to_csv("pv_disc_26Apr20.csv", index = False)

In [28]:
crsp3.to_csv("crsp_disc_26Apr20.csv")

In [ ]:
#######test###########